# LSH Algorithm Improvement By Applying Bitmap Indexing

In [84]:
! pip3 install progressbar2
import argparse
import sys
sss
from os import listdir
from os.path import isfile, join
from typing import Dict, List, Optional, Tuple
import imagehash
from PIL import Image
import os, os.path
import cv2
from collections import Counter
import scipy as sp
import numpy as np # Import numpy library 
from skimage.feature import hog # Import Hog model to extract features
from sklearn.metrics import confusion_matrix # Import confusion matrix to evaluate the performance
import pandas as pd
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import col
from pyspark.conf import SparkConf
from pyspark.ml.feature import BucketedRandomProjectionLSH
from pyspark.sql import SparkSession
from sklearn.model_selection import train_test_split

In [85]:
imgs = []
y = []
file_size = []
k = 0
path = "./data/101_ObjectCategories" # Give the dataset path here

##  Data Preprocessing:
1. Load the images using cv2
2. Image resize
3. Feature extraction: BGR to Gray conversion 
4. Feature extraction: Histogram of Oriented Gradients(HOG)

In [86]:
folder = os.listdir(path) # from the given path get the file names such as accordion, airplanes etc..
for file in folder: # for every file name in the given path go inseide that directory and get the images
    subpath = os.path.join(path,file)  # Join the name of these files to the previous path 
    
    files = os.listdir(subpath) # Take these image names to a list called files
    j = 0
    for i in range(np.size(files)): # now we shall loop through these number of files
        
        im = cv2.imread(subpath+'/'+files[0+j]) # Read the images from this subpath
        
        imgs.append(im) # append all the read images to a list called imgs
        y.append(k) # generate a labe to every file and append it to labels list

        j += 1
        if (j == (np.size(files))):
            file_size.append(j)
   
    k += 1
     
y = np.array(y).tolist()
ix = []
for index, item in enumerate(imgs):
    if (np.size(item) == 1):
        ix.append(index)
        del imgs[index]
        
for index, item in enumerate(y):
    for v in range(np.size(ix)):
        if (index == ix[v]):
            del y[index]
        
y = np.array(y).astype(np.float64) 

# Function to convert an image from color to grayscale
def rgb2gray(rgb):
    gray = cv2.cvtColor(rgb, cv2.COLOR_BGR2GRAY)
    return gray

def resize_(image):
    u = cv2.resize(image,(256,256))
    return u

def fd_hog(image):
    fd = hog(image, orientations=8, pixels_per_cell=(64, 64),
                        cells_per_block=(1, 1))
    return fd

In [87]:
a=[]
import progressbar
with progressbar.ProgressBar(max_value=len(imgs)) as bar:
    i=1
    for img in imgs:
        b=resize_(img)
        c=rgb2gray(b)   
        d=fd_hog(c)
        a.append(d)
        bar.update(i)
        i+=1

/Users/fudonghuang/anaconda3/lib/python3.7/site-packages/skimage/feature/_hog.py:150: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15. To supress this message specify explicitly the normalization method.
  skimage_deprecation)
100% (9176 of 9176) |####################| Elapsed Time: 0:01:33 Time:  0:01:33


## Split data
Split the data to training and validation data. We choose 70% for training and 30% for validation purposes.

In [79]:
%%time
# append 'label' and 'id' to the last two colunms
df = pd.DataFrame(a)
df['lable'] = y
id_ = np.arange(1,len(df)+1,1)
df['id'] = id_
X = df.values

CPU times: user 11.4 s, sys: 1.14 s, total: 12.6 s
Wall time: 13 s


In [80]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

## Using PySpark to retrieve similar images

In [82]:
spark = SparkSession.builder \
     .master("local") \
     .appName("Image Retrieval") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

In [83]:

Train = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_train)
Train_df = spark.createDataFrame(Train,schema=['id','label',"features"])

In [66]:
Test = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_test)
Test_df = spark.createDataFrame(Test,schema=['id','label',"features"])

In [67]:
Train_df.show(n = 2)

+----+-----+--------------------+
|  id|label|            features|
+----+-----+--------------------+
|8556|   96|[0.08273427838721...|
|7380|   91|[0.18656403041380...|
+----+-----+--------------------+
only showing top 2 rows



# !!!!! Skip以下代码直接运行最后一行 !!!!!

In [68]:
## skip以下代码直接运行最后一行

In [69]:
brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes",bucketLength=2,numHashTables=3)
model = brp.fit(Train_df)
print("The hashed dataset where hashed values are stored in the column 'hashes':")
model.transform(Train_df).show()

The hashed dataset where hashed values are stored in the column 'hashes':
+----+-----+--------------------+--------------------+
|  id|label|            features|              hashes|
+----+-----+--------------------+--------------------+
|8556|   96|[0.08273427838721...|[[0.0], [-1.0], [...|
|7380|   91|[0.18656403041380...|[[0.0], [-1.0], [...|
|2502|   29|[0.0,0.0,0.0,0.0,...|[[0.0], [-1.0], [...|
|6856|   88|[0.0,0.0,0.0,0.0,...|[[0.0], [-1.0], [...|
|7372|   91|[0.10931124019901...|[[-1.0], [-1.0], ...|
|8648|   96|[0.16685540296379...|[[-1.0], [-1.0], ...|
|4339|   60|[0.23326898398209...|[[-1.0], [-1.0], ...|
|2919|   37|[0.21724057435559...|[[0.0], [-1.0], [...|
|8480|   96|[0.18858223248917...|[[-1.0], [-1.0], ...|
| 872|   14|[0.20419114050679...|[[-1.0], [-1.0], ...|
|4193|   57|[0.02339184534831...|[[0.0], [-1.0], [...|
|3573|   47|[0.24555615498495...|[[-1.0], [-1.0], ...|
|3828|   51|[0.22501145437093...|[[-1.0], [-1.0], ...|
|1307|   18|[0.78224981011525...|[[-1.0], [-1.

In [70]:
key = Vectors.dense(X_test[0][0:-2])

In [71]:
key

DenseVector([0.1929, 0.1639, 0.1249, 0.1275, 0.1306, 0.1021, 0.107, 0.0511, 0.2256, 0.1419, 0.1972, 0.0714, 0.0824, 0.1086, 0.1092, 0.0637, 0.2685, 0.0748, 0.0962, 0.096, 0.1482, 0.0596, 0.1605, 0.0962, 0.1438, 0.1013, 0.1286, 0.1175, 0.1267, 0.0977, 0.1417, 0.1426, 0.2027, 0.0975, 0.1201, 0.0734, 0.1839, 0.0982, 0.114, 0.1102, 0.2413, 0.1078, 0.1386, 0.0663, 0.1333, 0.0564, 0.1588, 0.0974, 0.2562, 0.1078, 0.1244, 0.0414, 0.1471, 0.0768, 0.1113, 0.1352, 0.2795, 0.101, 0.1085, 0.0307, 0.0738, 0.0572, 0.1594, 0.1898, 0.2584, 0.0944, 0.0995, 0.0755, 0.1557, 0.0719, 0.1319, 0.1127, 0.1515, 0.1131, 0.1042, 0.08, 0.221, 0.1214, 0.1673, 0.0415, 0.1182, 0.0496, 0.1877, 0.0881, 0.2176, 0.0862, 0.1686, 0.084, 0.2344, 0.0737, 0.0918, 0.1633, 0.1874, 0.0679, 0.1073, 0.0741, 0.1205, 0.0678, 0.1401, 0.1653, 0.2274, 0.0882, 0.1008, 0.0899, 0.6684, 0.1292, 0.0235, 0.0125, 0.0116, 0.012, 0.0259, 0.117, 0.2501, 0.0802, 0.1005, 0.11, 0.0905, 0.0789, 0.139, 0.1509, 0.2591, 0.1225, 0.1015, 0.0716, 0.1486, 

In [72]:
X_test[0][-2]

16.0

In [73]:
print("Approximately searching Train_df for 2 nearest neighbors of the key:")
model.approxNearestNeighbors(Train_df, key, 5).show()

Approximately searching Train_df for 2 nearest neighbors of the key:
+----+-----+--------------------+--------------------+------------------+
|  id|label|            features|              hashes|           distCol|
+----+-----+--------------------+--------------------+------------------+
| 837|   13|[0.22847103669824...|[[0.0], [-1.0], [...|  3.82312494791629|
| 214|    3|[0.13452117699252...|[[0.0], [-1.0], [...| 3.887260607667535|
|3414|   45|[0.22119181409012...|[[0.0], [-1.0], [...|3.9091392577607724|
| 823|   13|[0.15263045832217...|[[0.0], [-1.0], [...|3.9191348209390697|
|5964|   81|[0.20823272783828...|[[-1.0], [-1.0], ...|3.9286809047501743|
+----+-----+--------------------+--------------------+------------------+



In [74]:
# result_id = result.select('label',).collect()
# result_id[0].label

In [75]:
# print("Approximately joining Train_df and Test_df on Euclidean distance smaller than 1:")
# model.approxSimilarityJoin(Train_df, Test_df, 1.1, distCol="EuclideanDistance")\
#     .select(col("datasetA.id").alias("Train_df"),
#             col("datasetB.id").alias("Test_df"),
#             col("EuclideanDistance")).show(30)

In [76]:
accuracy = 0
numOfNeighbor = 5
numOfTest= 5
accList = []
with progressbar.ProgressBar(max_value=numOfTest) as bar:
    for i in range(0, numOfTest):
        Catg = X_test[i][-2]
        key = Vectors.dense(X_test[i][0:-2])
        result = model.approxNearestNeighbors(Train_df, key, numOfNeighbor)
        temp = Counter([int(row['label']) for row in result.collect()])
        if  Catg in temp:
            accuracy += temp.get(Catg)/ numOfNeighbor
            accList.append(temp.get(Catg)/ numOfNeighbor)
        else:
            accList.append(0)
        bar.update(i)
    accuracy /= numOfTest

100% (5 of 5) |##########################| Elapsed Time: 0:00:18 Time:  0:00:18


In [77]:
accuracy

0.0

In [78]:
print(accList)

[0, 0, 0, 0, 0]


# 运行下面一行 

In [91]:
def getBestPerformance(a, bucketLength,numHashTables,numOfNeighbor):
    
    df = pd.DataFrame(a)
    df['lable'] = y
    id_ = np.arange(1,len(df)+1,1)
    df['id'] = id_
    X = df.values
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

    spark = SparkSession.builder \
     .master("local") \
     .appName("Image Retrieval") \
     .config("spark.some.config.option", "some-value") \
     .getOrCreate()

    Train = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_train)
    Train_df = spark.createDataFrame(Train,schema=['id','label',"features"])
    Test = map(lambda x: (int(x[-1]),int(x[-2]),Vectors.dense(x[:-2])), X_test)
    Test_df = spark.createDataFrame(Test,schema=['id','label',"features"])

    brp = BucketedRandomProjectionLSH(inputCol="features", outputCol="hashes", 
                                      bucketLength=bucketLength,numHashTables=numHashTables)
    
    
    model = brp.fit(Train_df)
    model.transform(Train_df)
    ## run test dataset 
    numOfTest = 100
    accuracy = 0
    with progressbar.ProgressBar(max_value = numOfTest) as bar:
        for i in range(0, numOfTest):
            Catg = X_test[i][-2]
            key = Vectors.dense(X_test[i][0:-2])
            result = model.approxNearestNeighbors(Train_df, key, numOfNeighbor)
            temp = Counter([int(row['label']) for row in result.collect()])
            if  Catg in temp:
                accuracy += temp.get(Catg)/ numOfNeighbor
            bar.update(i)
        accuracy /= numOfTest
    return accuracy

In [ ]:
bucketLengthList = [1.5, 8, 15]
numHashTablesList = [1, 2, 55, 100]
numOfNeighbor = [5]
strList = []
resList = []

for i in bucketLengthList:
    for j in numHashTablesList:
        for k in numOfNeighbor:
            result = getBestPerformance(a, i, j, k)
            resList.append(result)
            resStr = "bucketLen:" + str(i) + "  #Hashtable:" + str(j) + "  #Neighbor:" + str(k) + "  Acc:" + str(result)
            print(resStr)
            strList.append(resStr)

100% (100 of 100) |######################| Elapsed Time: 0:00:27 Time:  0:00:27


bucketLen:1.5  #Hashtable:1  #Neighbor:5  Acc:0.35199999999999987


100% (100 of 100) |######################| Elapsed Time: 0:00:26 Time:  0:00:26


bucketLen:1.5  #Hashtable:2  #Neighbor:5  Acc:0.376


100% (100 of 100) |######################| Elapsed Time: 0:01:15 Time:  0:01:15


...

100% (100 of 100) |######################| Elapsed Time: 0:00:26 Time:  0:00:26


bucketLen:8  #Hashtable:1  #Neighbor:5  Acc:0.36399999999999993


100% (100 of 100) |######################| Elapsed Time: 0:00:26 Time:  0:00:26


bucketLen:8  #Hashtable:2  #Neighbor:5  Acc:0.444


 78% (78 of 100) |#################      | Elapsed Time: 0:00:58 ETA:   0:00:19

In [ ]:
strList

In [ ]:
resList

In [ ]:
# from matplotlib.pyplot import imshow
# imshow(imgs[4795])